In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.4 MB/s eta 0:00:00


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split, GridSearchCV, KFold

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor
import xgboost as xgb

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

from scipy.stats import norm
import numpy as np
import tensorflow as tf

In [6]:
url = "https://raw.githubusercontent.com/Tejanikhil/Shared-ASAG/main"
df = pd.read_csv(url + "/SemEvalData.csv")

In [7]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

# Download the stopwords corpus if not already downloaded
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
def extract_pos(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    pos_words = [word for word, pos in pos_tags if pos.startswith('VB') or pos.startswith('JJ') or pos.startswith('NN')]
    return ' '.join(pos_words)

def process_dataset(text):
    lower_cased = text.lower()
    tokens = nltk.word_tokenize(lower_cased)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    preprocessed = ' '.join(filtered_tokens)
    return preprocessed

In [9]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(df.loc[:,["Actual Answer","Student Answer"]], df["Grade"], test_size=0.1, random_state=42)

In [10]:
TRAIN = train_X
TRAIN["LABELS"] = train_y
TEST = test_X
TEST["LABELS"] = test_y

In [11]:
train_X["Actual Answer"] = train_X["Actual Answer"].apply(extract_pos).apply(process_dataset)
train_X["Student Answer"] = train_X["Student Answer"].apply(extract_pos).apply(process_dataset)

test_X["Actual Answer"] = test_X["Actual Answer"].apply(extract_pos).apply(process_dataset)
test_X["Student Answer"] = test_X["Student Answer"].apply(extract_pos).apply(process_dataset)

In [12]:
print(TRAIN.shape)
TRAIN = TRAIN.replace("", pd.NA).dropna()
print(TRAIN.shape)
TEST = TEST.replace("", pd.NA).dropna()

(2197, 3)
(2184, 3)


In [13]:
train_X = TRAIN.drop("LABELS", axis = 1)
train_y = TRAIN.iloc[:,-1]

In [14]:
sentences = list(train_X["Actual Answer"]) + list(train_X["Student Answer"])

In [15]:
rm -rf ~/.cache/huggingface/transformers/bert-base-uncased

In [16]:
from flair.embeddings import FlairEmbeddings
from flair.data import Sentence
import flair

In [17]:
flair_forward = flair.embeddings.FlairEmbeddings('multi-forward')
flair_backward = flair.embeddings.FlairEmbeddings('multi-backward')
flair_en_forward = flair.embeddings.FlairEmbeddings('en-forward')

2023-05-28 14:15:32,939 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-jw300-forward-v0.1.pt not found in cache, downloading to /tmp/tmpyl3jovf1


100%|██████████| 165M/165M [00:14<00:00, 11.8MB/s]

2023-05-28 14:15:48,288 copying /tmp/tmpyl3jovf1 to cache at /root/.flair/embeddings/lm-jw300-forward-v0.1.pt


2023-05-28 14:15:48,536 removing temp file /tmp/tmpyl3jovf1
2023-05-28 14:15:57,124 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-jw300-backward-v0.1.pt not found in cache, downloading to /tmp/tmpvd5kytpi


100%|██████████| 165M/165M [00:14<00:00, 12.1MB/s]

2023-05-28 14:16:12,058 copying /tmp/tmpvd5kytpi to cache at /root/.flair/embeddings/lm-jw300-backward-v0.1.pt


2023-05-28 14:16:12,256 removing temp file /tmp/tmpvd5kytpi
2023-05-28 14:16:13,726 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-forward-0.4.1.pt not found in cache, downloading to /tmp/tmpdjeyx84p


100%|██████████| 69.7M/69.7M [00:06<00:00, 10.7MB/s]

2023-05-28 14:16:21,253 copying /tmp/tmpdjeyx84p to cache at /root/.flair/embeddings/news-forward-0.4.1.pt
2023-05-28 14:16:21,337 removing temp file /tmp/tmpdjeyx84p


In [18]:
flair_news_forward = flair.embeddings.FlairEmbeddings('news-forward')
flair_news_backward = flair.embeddings.FlairEmbeddings('news-backward')

2023-05-28 14:16:55,804 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmpsi0vvsra


100%|██████████| 69.7M/69.7M [00:06<00:00, 10.6MB/s]

2023-05-28 14:17:04,002 copying /tmp/tmpsi0vvsra to cache at /root/.flair/embeddings/news-backward-0.4.1.pt


2023-05-28 14:17:04,103 removing temp file /tmp/tmpsi0vvsra


In [19]:
train_Embeddings_df = pd.DataFrame({"Student_Answer_Embeddings":train_X["Student Answer"], "Reference_Answer_Embeddings":train_X["Actual Answer"]})
test_Embeddings_df = pd.DataFrame({"Student_Answer_Embeddings":test_X["Student Answer"], "Reference_Answer_Embeddings":test_X["Actual Answer"]})

In [21]:
def get_flair_multi_forward_embeddings(text):
  sentence = Sentence(text)
  flair_forward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [22]:
def get_flair_multi_backward_embeddings(text):
  sentence = Sentence(text)
  flair_backward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [23]:
def get_flair_news_forward_embeddings(text):
  sentence = Sentence(text)
  flair_news_forward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [24]:
def get_flair_news_backward_embeddings(text):
  sentence = Sentence(text)
  flair_news_backward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [25]:
def get_flair_en_forward_embeddings(text):
  sentence = Sentence(text)
  flair_en_forward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [26]:
get_flair_en_forward_embeddings("I am a good boy")

array([ 0.00305832, -0.00205429,  0.0675773 , ..., -0.00094282,
       -0.02816246,  0.0060261 ], dtype=float32)

In [27]:
train_Embeddings_df["Student_Answer_Embeddings"] = train_Embeddings_df["Student_Answer_Embeddings"].apply(get_flair_en_forward_embeddings)
train_Embeddings_df["Reference_Answer_Embeddings"] = train_Embeddings_df["Reference_Answer_Embeddings"].apply(get_flair_en_forward_embeddings)

test_Embeddings_df["Student_Answer_Embeddings"] = test_Embeddings_df["Student_Answer_Embeddings"].apply(get_flair_en_forward_embeddings)
test_Embeddings_df["Reference_Answer_Embeddings"] = test_Embeddings_df["Reference_Answer_Embeddings"].apply(get_flair_en_forward_embeddings)

In [28]:
train_Embeddings_df

,Student_Answer_Embeddings,Reference_Answer_Embeddings
199,"[0.014949691, -0.00028209697, 0.26412484, -0.1...","[0.004128535, 0.00040851586, 0.034285467, 0.00..."
229,"[-0.0069785835, -0.00028074757, 0.08945726, 0....","[-0.002298493, -0.00022442442, 0.051880054, 0...."
521,"[0.00086842314, 6.345646e-05, 0.050162856, 0.0...","[-2.7866529e-06, -4.322462e-05, 0.07735125, 0...."
651,"[-0.004799666, 9.13227e-05, 0.04184654, 0.0069...","[-0.0019756893, 0.0005305293, 0.13633445, 0.02..."
450,"[0.001094765, 0.0007270078, 0.075637236, 0.003...","[0.00039971713, 9.8635675e-05, 0.02593081, 0.0..."
...,...,...
1638,"[0.0060006334, 0.0002594909, 0.056259908, 0.00...","[-0.002699483, 0.005450536, 0.03229649, 0.0054..."
1095,"[0.00491989, -0.00038775906, 0.026758801, 0.02...","[0.00081963366, 3.7649454e-05, 0.12028385, 0.0..."
1130,"[-0.0008440845, -0.00043202046, 0.07099004, 0....","[0.0017804197, -6.2974505e-05, 0.06401146, 0.0..."
1294,"[0.0054848, -0.000121210374, 0.058077753, 0.00...","[-0.001415609, 0.00055790914, 0.0696167, -0.00..."


In [29]:
train_X["Subtracted_Embeddings"] = train_Embeddings_df.apply(lambda x:x["Student_Answer_Embeddings"]-x["Reference_Answer_Embeddings"], axis = 1)
test_X["Subtracted_Embeddings"] = test_Embeddings_df.apply(lambda x:x["Student_Answer_Embeddings"]-x["Reference_Answer_Embeddings"], axis = 1)

In [30]:
train_df = pd.DataFrame(list(train_X["Subtracted_Embeddings"]))
test_df = pd.DataFrame(list(test_X["Subtracted_Embeddings"]))

In [31]:
train_df.to_csv("flair_en_forward_embeddings.csv")

In [32]:
pca = PCA(n_components=100) 
pca.fit_transform(train_df)
principal_components = pca.transform(train_df)
train_df_pca = pd.DataFrame(data=principal_components)

In [33]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()
LR_model.fit(train_df, train_y)

LinearRegression()

In [34]:
preds = LR_model.predict(test_df)
y_preds = [round(y_val, 1) for y_val in preds]
actuals = test_y
y_true = [round(y_val, 1) for y_val in actuals]

In [35]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
LR_mae = mean_absolute_error(y_true, np.round(y_preds, 1))
LR_mse = mean_squared_error(y_true, np.round(y_preds, 1))
LR_rmse = mean_squared_error(y_true, np.round(y_preds, 1), squared=False)
LR_mape = mean_absolute_percentage_error(y_true, y_preds, multioutput = "uniform_average")
LR_r2 = r2_score(y_true, y_preds)

In [36]:
print("Mean Absolute Error (MAE): ", np.round(LR_mae, 2))
print("Mean Squared Error (MSE): ", np.round(LR_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(LR_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(LR_mape, 2))
print("R2 Score: ", np.round(LR_r2, 2))

Mean Absolute Error (MAE):  52.33
Mean Squared Error (MSE):  7217.3
Root Mean Squared Error (RMSE):  84.95
Mean Absolute Percentage Error (MAPE):  5898796341521727.0
R2 Score:  -3595.21


In [37]:
from sklearn import neighbors
rmse_val = []
mae_val = []
mse_val = []
r2_val = []
for K in range(2,20):
    knn_model = neighbors.KNeighborsRegressor(n_neighbors = K)
    knn_model.fit(train_df, train_y)  #fit the model
    pred=knn_model.predict(test_df) #make prediction on test set
    
    mae = mean_absolute_error(y_true, np.round(pred, 1))
    mae_val.append(mae)
    
    mse = mean_squared_error(y_true, np.round(pred, 1))
    mse_val.append(mse)
    
    rmse = mean_squared_error(y_true, np.round(pred, 1), squared=False)
    rmse_val.append(rmse)
    
    r2 = r2_score(y_true, pred)
    r2_val.append(r2)

In [38]:
y_true = test_y
knn_preds = knn_model.predict(test_df)

In [39]:
min_val = min(r2_val)
index = r2_val.index(min_val)
knn_r2 = min_val
knn_rmse = rmse_val[index]
knn_mae = mae_val[index]
knn_mse = mse_val[index]

In [40]:
print("Mean Absolute Error (MAE): ", np.round(knn_mae, 2))
print("Mean Squared Error (MSE): ", np.round(knn_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(knn_rmse, 2))
print("R2 Score: ", np.round(knn_r2, 2))

Mean Absolute Error (MAE):  1.17
Mean Squared Error (MSE):  2.81
Root Mean Squared Error (RMSE):  1.68
R2 Score:  -0.39


In [41]:
ada_reg = AdaBoostRegressor(n_estimators = 100, learning_rate=0.5)

ada_reg.fit(train_df, train_y)

ada_reg_pred = ada_reg.predict(pd.DataFrame(test_df))
ada_reg_mae = mean_absolute_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mse = mean_squared_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mape = mean_absolute_percentage_error(test_y, np.round(ada_reg_pred,1), multioutput = "uniform_average")
ada_reg_rmse = mean_squared_error(test_y, np.round(ada_reg_pred, 1), squared=False)
ada_reg_r2 = r2_score(test_y, ada_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(ada_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(ada_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(ada_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(ada_reg_mape, 2))
print("R2 Score: ", np.round(ada_reg_r2, 2))

Mean Absolute Error (MAE):  1.4
Mean Squared Error (MSE):  2.44
Root Mean Squared Error (RMSE):  1.56
Mean Absolute Percentage Error (MAPE):  476094817750595.5
R2 Score:  -0.23


In [42]:
gb_reg = GradientBoostingRegressor(n_estimators = 50, max_depth=5)

gb_reg.fit(train_df, train_y)

gb_reg_pred = gb_reg.predict(pd.DataFrame(test_df))
gb_reg_mae = mean_absolute_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mse = mean_squared_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mape = mean_absolute_percentage_error(test_y, np.round(gb_reg_pred,1), multioutput = "uniform_average")
gb_reg_rmse = mean_squared_error(test_y, np.round(gb_reg_pred, 1), squared=False)
gb_reg_r2 = r2_score(test_y, gb_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(gb_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(gb_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(gb_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(gb_reg_mape, 2))
print("R2 Score: ", np.round(gb_reg_r2, 2))

Mean Absolute Error (MAE):  1.07
Mean Squared Error (MSE):  1.84
Root Mean Squared Error (RMSE):  1.36
Mean Absolute Percentage Error (MAPE):  481609429539212.56
R2 Score:  0.07


In [43]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', seed = 123)

xg_reg.fit(train_df, train_y)

# print("Best parameters: ", xg_boost_grid_search.best_params_)
# print("Best score: ", xg_boost_grid_search.best_score_)
xg_reg_pred = xg_reg.predict(pd.DataFrame(test_df))
xg_reg_mae = mean_absolute_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mse = mean_squared_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mape = mean_absolute_percentage_error(test_y, np.round(xg_reg_pred,1), multioutput = "uniform_average")
xg_reg_rmse = mean_squared_error(test_y, np.round(xg_reg_pred, 1), squared=False)
xg_reg_r2 = r2_score(test_y, xg_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(xg_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(xg_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(xg_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(xg_reg_mape, 2))
print("R2 Score: ", np.round(xg_reg_r2, 2))

Mean Absolute Error (MAE):  1.06
Mean Squared Error (MSE):  2.04
Root Mean Squared Error (RMSE):  1.43
Mean Absolute Percentage Error (MAPE):  479771230019684.6
R2 Score:  -0.03
